In [1]:
"""
Source: https://www.pinecone.io/learn/series/langchain/langchain-tools/
"""
None

In [2]:
from langchain.tools import BaseTool
from math import pi
from typing import Union


class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        return float(radius) * 2.0 * pi

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")


class GetClientSpentHistoryTool(BaseTool):
    name = "Client Spent History"
    description = (
        "use this tool to get the historical spent from a client, "
        "also called advertiser or parent company"
    )

    def _run(self, client: str):
        return {
            "date": [
                "2020-jan",
                "2020-fev",
                "2020-mar",
                "2020-apr",
                "2020-may",
                "2020-jun",
            ],
            "spent": [100, 120, 150, 100, 90, 100],
        }

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

In [79]:
from langchain.agents import initialize_agent

tools = [CircumferenceTool(), GetClientSpentHistoryTool()]

# initialize agent with tools
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    memory=conversational_memory,
)

In [5]:
agent("can you calculate the circumference of a circle that has a radius of 7.81mm")
# Correct Answer: 49.0717



> Entering new AgentExecutor chain...
{
    "action": "Circumference calculator",
    "action_input": "7.81"
}
Observation: 49.071677249072565
Thought:{
    "action": "Final Answer",
    "action_input": "The circumference of a circle with a radius of 7.81mm is approximately 49.07mm."
}

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81mm',
 'chat_history': [],
 'output': 'The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'}

In [8]:
agent("How much did Amazon, spent in January?")



> Entering new AgentExecutor chain...
{
    "action": "Client Spent History",
    "action_input": "Amazon"
}
Observation: {'date': ['2020-jan', '2020-fev', '2020-mar', '2020-apr', '2020-may', '2020-jun'], 'spent': [100, 120, 150, 100, 90, 100]}
Thought:{
    "action": "Final Answer",
    "action_input": "Amazon spent $100 in January."
}

> Finished chain.


{'input': 'How much did Amazon, spent in January?',
 'chat_history': [HumanMessage(content='can you calculate the circumference of a circle that has a radius of 7.81mm', additional_kwargs={}, example=False),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.', additional_kwargs={}, example=False),
  HumanMessage(content='How much the client Amazon, spent in January?', additional_kwargs={}, example=False),
  AIMessage(content='The client Amazon spent $100 in January.', additional_kwargs={}, example=False),
  HumanMessage(content='How much did Amazon, spent in January?', additional_kwargs={}, example=False),
  AIMessage(content='Amazon spent $100 in January.', additional_kwargs={}, example=False)],
 'output': 'Amazon spent $100 in January.'}

In [72]:
import pprint
from collections.abc import Iterable

pp = pprint.PrettyPrinter(width=41, compact=True)


def print_obj(obj, level=0, prev_obj=""):
    try:
        for x, y in obj:
            y = (
                y
                if x != "openai_api_key"
                else "Not Today ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
            )
            if isinstance(y, Iterable) and not isinstance(y, str):
                print_obj(y, level + 1, f"{prev_obj}.{x}")
            else:
                print(f"{'||'*level} {prev_obj}.{x}:: >```{y}```<")
    except ValueError:
        for i, y in enumerate(obj):
            if isinstance(y, Iterable) and not isinstance(y, str):
                print_obj(y, level + 1, f"{prev_obj}[{i}]")
            else:
                print(f"{'||'*level} {prev_obj}[{i}]: >```{y}```<")


print_obj(agent.agent)

|| .llm_chain.memory:: >```None```<
|| .llm_chain.callbacks:: >```None```<
|| .llm_chain.callback_manager:: >```None```<
|| .llm_chain.verbose:: >```False```<
|| .llm_chain.tags:: >```None```<
|| .llm_chain.metadata:: >```None```<
|||||| .llm_chain.prompt.input_variables[0]: >```input```<
|||||| .llm_chain.prompt.input_variables[1]: >```chat_history```<
|||||| .llm_chain.prompt.input_variables[2]: >```agent_scratchpad```<
|||| .llm_chain.prompt.output_parser:: >```None```<
|||||||| .llm_chain.prompt.messages.('prompt', PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversa

In [77]:
print(agent.agent.llm_chain.prompt.messages[2].prompt.template)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Circumference calculator: use this tool when you need to calculate a circumference using the radius of a circle
> Client Spent History: use this tool to get the historical spent from a client, also called advertiser or parent company

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string, \ The action to take. Must be one of Circumference calculator, Client Spent History
    "action_input": string \ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answ